In [ ]:
from functools import partial
import geopyspark as gps
import numpy as np
import fiona
import json
import pyproj

from pyspark import SparkContext
from colortools import Color

from shapely.geometry import mapping, shape
from shapely.ops import transform

from folium import Map, TileLayer, GeoJson

In [ ]:
conf = gps.geopyspark_conf(master="local[*]", appName="NLCD Viewer")
sc = SparkContext(conf=conf)

In [ ]:
catalog_uri = "s3://azavea-datahub/catalog"
layer_name = "nlcd-2011-epsg3857"

## Viewing NLCD

In [ ]:
nlcd_cmap = gps.ColorMap.nlcd_colormap()
nlcd_tms_server = gps.TMS.build((catalog_uri, layer_name), display=nlcd_cmap)
nlcd_tms_server.bind('0.0.0.0')
nlcd_tms_server.url_pattern

In [ ]:
m = Map(tiles='Stamen Terrain', location=[37.1, -95.7], zoom_start=4)
TileLayer(tiles=nlcd_tms_server.url_pattern, attr='GeoPySpark Tiles').add_to(m)
m

## Viewing reclassified tiles

This example shows how to do custom, on-the-fly display from an existing catalog using a callback to a Python rendering function.  This method is much slower than using color maps.  Please be patient during map display/zooming.

In [ ]:
import struct
from PIL import Image

def from_color_get_component(i):
    def fn(x):
        split = struct.Struct(">I").pack
        r,g,b,a = split(x & 0xffffffff)
        return np.array([r,g,b,a], dtype='uint8')[i]
    return fn

def render_tile(tile):
    rr = np.vectorize(from_color_get_component(0))(tile)
    gg = np.vectorize(from_color_get_component(1))(tile)
    bb = np.vectorize(from_color_get_component(2))(tile)
    aa = np.vectorize(from_color_get_component(3))(tile)
    return Image.fromarray(np.dstack([rr, gg, bb, aa]), mode='RGBA')

In [ ]:
from PIL import Image
import struct

def render_cultivated(tile):
    # NLCD codes in the 80's are Planted/Cultivated
    # See https://www.mrlc.gov/nlcd11_leg.php
    colorize = np.vectorize(lambda x: 0x7110b2aa if ((80 <= x) & (x < 90)) else 0x00000000)
    return render_tile(colorize(tile[0][0]))

In [ ]:
custom_nlcd_tms_server = gps.TMS.build((catalog_uri, layer_name), display=render_cultivated)
custom_nlcd_tms_server.bind('0.0.0.0')
custom_nlcd_tms_server.url_pattern

In [ ]:
m = Map(tiles='Stamen Terrain', location=[37.1, -95.7], zoom_start=4)
TileLayer(tiles=custom_nlcd_tms_server.url_pattern, attr='GeoPySpark Tiles').add_to(m)
m

## Chattanooga geometry

In [ ]:
!curl -o /tmp/mask.json https://s3.amazonaws.com/chattademo/chatta_mask.json

In [ ]:
txt = open('/tmp/mask.json').read()
js = json.loads(txt)
geom = shape(js)
center = geom.centroid
chatta_center = [center.y, center.x] # Location in lat/long

In [ ]:
GeoJson('/tmp/mask.json', name='Chattanooga').add_to(m)

In [ ]:
m.location = chatta_center
m.zoom_start = 8
m

## Fetching an RDD of NLCD masked to Chattanooga

In [ ]:
project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:3857'))

chatta_poly = transform(project, geom)

In [ ]:
query_rdd = gps.query(catalog_uri,
                      layer_name,
                      12,
                      query_geom=chatta_poly)

In [ ]:
masked = query_rdd.mask([chatta_poly])
masked_tms_server = gps.TMS.build(masked.pyramid(), display=nlcd_cmap)
masked_tms_server.bind('0.0.0.0')

In [ ]:
chatta_map = Map(tiles='Stamen Terrain', location=chatta_center, zoom_start=8)
TileLayer(tiles=masked_tms_server.url_pattern, attr='GeoPySpark Tiles').add_to(chatta_map)
GeoJson('/tmp/mask.json', name='Chattanooga').add_to(chatta_map)
chatta_map

## Reclassifying an RDD

In [ ]:
reclassified = masked.reclassify({0: 1, 80: 2, 90: 1},
                                 int,
                                 gps.ClassificationStrategy.GREATER_THAN_OR_EQUAL_TO).repartition(150)

In [ ]:
colors = gps.get_colors_from_colors(
    [Color("#CA9146FF"), Color("#00FFAA88")])

breaks = {
    1: colors[0],
    2: colors[1]
}

reclassified_cmap = gps.ColorMap.build(breaks)

In [ ]:
reclassified_tms_server = gps.TMS.build(reclassified.pyramid(), display=reclassified_cmap)
reclassified_tms_server.bind('0.0.0.0')

In [ ]:
reclass_map = Map(tiles='Stamen Terrain', location=chatta_center, zoom_start=8)
TileLayer(tiles=reclassified_tms_server.url_pattern, attr='GeoPySpark Tiles').add_to(reclass_map)
GeoJson('/tmp/mask.json', name='Chattanooga').add_to(reclass_map)
reclass_map

## Saving the reclassified layer locally

In [ ]:
local_catalog_uri = "file:///tmp/catalog"
local_layer_name = "cultivated-land-cover"

In [ ]:
for layer in reclassified.pyramid().levels.values():
    gps.write(local_catalog_uri, local_layer_name, layer)

## Viewing the local Layer

In [ ]:
nlcd_tms_server.unbind()
custom_nlcd_tms_server.unbind()
masked_tms_server.unbind()
reclassified_tms_server.unbind()

In [ ]:
local_tms_server = gps.TMS.build((local_catalog_uri, local_layer_name), reclassified_cmap)
local_tms_server.bind('0.0.0.0')

In [ ]:
local_map = Map(tiles='Stamen Terrain', location=chatta_center, zoom_start=8)
TileLayer(tiles=local_tms_server.url_pattern, attr='GeoPySpark Tiles').add_to(local_map)
GeoJson('/tmp/mask.json', name='Chattanooga').add_to(local_map)
local_map

In [ ]:
layers = [gps.query(local_catalog_uri, local_layer_name, x) for x in range(0, 11)]

In [ ]:
read_in_pyramid = gps.Pyramid(layers)

In [ ]:
# This cannot display as well
server = gps.TMS.build(read_in_pyramid, reclassified_cmap)
server.bind('0.0.0.0')

In [ ]:
rdd_map = Map(tiles='Stamen Terrain', location=chatta_center, zoom_start=8)
TileLayer(tiles=server.url_pattern, attr='GeoPySpark Tiles').add_to(rdd_map)
GeoJson('/tmp/mask.json', name='Chattanooga').add_to(rdd_map)
rdd_map